# Подключение к модели

##### Данный этап выполнялся локально на ПК через llama.cpp.

In [ ]:
# установка нужных библиотек
!pip install langchain faiss-cpu sentence-transformers requests

#### Эмбеддинг-модель: intfloat/multilingual-e5-large


#### LLM: yandex/YandexGPT-5-Lite-8B-instruct

In [ ]:
import requests
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer

DEADLINE_SIMILARITY = 0.95      # ограничение на близость фрагментов документов (показатель высокий из-за особенностей эмбеддинг-модели)

# Инициализируем модель встраивания
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": "cpu"}
)

# Загружаем FAISS индекс
db = FAISS.load_local(
    r"C:\Users\zvere\Downloads\faiss_index_cosine",
    embedding_model,
    allow_dangerous_deserialization=True
)

# Инициализируем токенизатор
MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# URL сервера LLM
LLM_SERVER_URL = "http://localhost:8080/completion"

# Функция для создания промпта
def create_prompt(query, context=None, history=None):
    messages = []
    
    # Добавляем историю диалога
    if history:
        for msg in history:
            messages.append({"role": "user", "content": msg['user']})
            if 'assistant' in msg:
                messages.append({"role": "assistant", "content": msg['assistant']})
    
    # Формируем основное сообщение
    if context:
        context_str = "Фрагменты из документов:\n"
        for i, doc in enumerate(context, 1):
            context_str += f"[Фрагмент {i}] Документ-источник: {doc.metadata['document']}\nТекст: {doc.page_content}\n[Конец фрагмента {i}]\n"
        messages.append({
            "role": "user",
            "content": (
                "Ты - **юридический ассистент**.\n"
                "**Следуй этим шагам для ответа**:\n"
                "1. Проанализируй предоставленные фрагменты из документов. Фрагменты могут быть обрывистыми.\n"
                "2. Представь, что я не предоставлял тебе предоставленные фрагменты, но ты их изучал.\n"
                "3. Если фрагменты содержат релевантную информацию, используй её для ответа! Сочетай информацию из них со своими знаниями.\n"
                "4. Указывай в ответе название документа-источника, если это законодательный документ.\n"
                "5. Если фрагменты не содержат релевантной информации или совсем не относятся к вопросу, отвечай на основе своих знаний.\n"
                "6. Отвечай подробно, чётко и профессионально по сути вопроса.\n"
                "7. Если вопрос не юридический, используй дружелюбный стиль, но оставайся профессиональным.\n"
                "8. Не используй в ответе формат markdown, пиши обычным текстом.\n\n"
                f"{context_str}\n**Вопрос**: {query}"
            )
        })
    else:
        messages.append({
            "role": "user",
            "content": (
                "Ты - **юридический ассистент**. "
                "Отвечай на вопрос максимально чётко и профессионально, даже если это не юридический вопрос, "
                "используя дружелюбный стиль. Не используй в ответе формат markdown, пиши обычным текстом.\n\n"
                f"**Вопрос**: {query}"
            )
        })
    
    # Применяем chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return prompt


# Функция для генерации ответа
def generate_response(query, user_chat_id, threshold=DEADLINE_SIMILARITY, top_k=3):
    try:
        # Получаем историю диалога для пользователя
        history = user_dialog_history.get(user_chat_id, [])
        
        # Ищем топ-k чанков с их оценками схожести
        results = db.similarity_search_with_score(query, k=top_k)

        # Печатаем все найденные чанки и их значения схожести
        print("Найденные релевантные чанки:")
        for i, (doc, l2_distance) in enumerate(results, 1):
            cosine_sim = 1 - (l2_distance ** 2) / 2
            print(f"Чанк {i}:")
            print(f"  Текст: {doc.page_content}")
            print(f"  Источник: {doc.metadata['document']}")
            print(f"  Косинусное сходство: {cosine_sim:.4f}")
        print("-" * 50)

        # Проверяем релевантность лучшего чанка
        context = None
        if results:
            relevant_chunks = [res[0] for res in results if (1 - (res[1] ** 2) / 2) >= threshold]
            if relevant_chunks:
                context = relevant_chunks
                print(f"Контекст используется (лучшее косинусное сходство: {(1 - (results[0][1] ** 2) / 2):.4f})")
            else:
                print("Контекст не используется (ни один чанк не прошёл порог релевантности)")
        
        # Формируем промпт с историей
        prompt = create_prompt(query, context, history)
        
        # Выводим промпт для отладки
        print("Окончательный промпт, отправленный в модель:")
        print(prompt)
        print("-" * 50)
        
        # Отправляем запрос к LLM
        response = requests.post(LLM_SERVER_URL, json={
            "prompt": prompt,
            "max_tokens": 1024,
            "temperature": 0.3,
            "stop": ["</s>"]
        })
        
        if response.status_code == 200:
            # Удаляем специальные токены из ответа
            answer = response.json()["content"].replace("[NL]", "\n").replace("<s>", "").replace("</s>", "").strip()
            print(answer)
            
            # Обновляем историю диалога (если сообщений >= 10)
            if len(history) >= 10:
                history.pop(0)  # Удаляем самое старое сообщение
            history.append({"user": query, "assistant": answer})
            user_dialog_history[user_chat_id] = history
            
            return answer
        else:
            print(f"Ошибка при обращении к серверу LLM: {response.status_code}")
            return f"Ошибка при обращении к серверу LLM: {response.status_code}"
    except Exception as e:
        print(f"Ошибка: {str(e)}")
        return f"Ошибка: {str(e)}"

# Подключение к telegram-боту

**Замечание**: в боте также реализована возможность диалога с реальным человеком.

In [ ]:
# установка нужных библиотек
!pip install python-telegram-bot nest_asyncio

In [ ]:
import asyncio
import nest_asyncio
import requests
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import re

user_dialog_history = {}  # {user_chat_id: [{"user": "вопрос", "assistant": "ответ"}, ...]}

nest_asyncio.apply()

# Конфигурация специалиста - ЗАМЕНИТЕ НА РЕАЛЬНЫЙ CHAT_ID СПЕЦИАЛИСТА!
SPECIALIST_CHAT_ID = "123456789"

# Глобальное хранилище для консультаций
active_consultations = {}  # {user_chat_id: specialist_chat_id}
user_info_db = {}  # {user_chat_id: {"name": str, "username": str}}

# для команды /start
async def start(update: Update, context):
    context.user_data['in_consultation'] = False
    await update.message.reply_text(
        "👋 Здравствуйте! Я Ваш личный юридический нейроассистент. Задайте мне вопрос!\n"
        "📞 Для консультации со специалистом используйте /consult\n"
        "🆕 Чтобы начать новый диалог со мной - /newdialog\n"
        "❗ Бот работает в экспериментальном режиме, и его функционирование происходит локально на ноутбуке одного из авторов проекта, из-за чего также могут быть небольшие задержки в получении ответов. Просим проявить терпение! Ответы вне консультаций с экспертом генерируются ИИ. Проверяйте важную информацию."
    )

# для команды /consult
async def start_consultation(update: Update, context):
    user_chat_id = update.message.chat_id
    user = update.message.from_user
    
    # Проверяем, не активна ли уже консультация
    if context.user_data.get('in_consultation', False):
        await update.message.reply_text("❗ У вас уже активна консультация со специалистом.")
        return
    
    # Сохраняем информацию о пользователе
    user_name = user.first_name + (" " + user.last_name if user.last_name else "")
    user_info_db[user_chat_id] = {
        "name": user_name,
        "username": user.username if user.username else "не указан"
    }
    
    # Устанавливаем флаг консультации
    context.user_data['in_consultation'] = True
    
    try:
        # Сохраняем связь пользователь-специалист
        active_consultations[user_chat_id] = SPECIALIST_CHAT_ID
        
        # Уведомляем пользователя
        await update.message.reply_text(
            "✅ Вы подключены к специалисту. Задайте ваш вопрос, "
            "и он ответит в ближайшее время.\n\n"
            "🔸 Для возврата к боту используйте /newdialog"
        )
        
        # Уведомляем специалиста
        await context.bot.send_message(
            chat_id=SPECIALIST_CHAT_ID,
            text=f"🔔 НОВАЯ КОНСУЛЬТАЦИЯ 🔔\n"
                 #f"👤 Пользователь: {user_name}\n"
                 #f"📱 Username: @{user_info_db[user_chat_id]['username']}\n"
                 f"🆔 ID: {user_chat_id}\n\n"
                 f"✉️ Чтобы ответить, введите:\n"
                 f"   /to_{user_chat_id} Ваш ответ"
        )
        
    except Exception as e:
        print(f"Ошибка при подключении к специалисту: {e}")
        await update.message.reply_text(
            "❌ Не удалось подключиться к специалисту. "
            "Пожалуйста, попробуйте позже или свяжитесь другим способом."
        )
        context.user_data['in_consultation'] = False
        if user_chat_id in active_consultations:
            del active_consultations[user_chat_id]

# для команды /newdialog
async def new_dialog(update: Update, context):
    user_chat_id = update.message.chat_id
    
    if context.user_data.get('in_consultation', False):
        # Завершаем консультацию
        context.user_data['in_consultation'] = False
        
        # Уведомляем специалиста
        if user_chat_id in active_consultations:
            try:
                await context.bot.send_message(
                    chat_id=active_consultations[user_chat_id],
                    text=f"🔴 КОНСУЛЬТАЦИЯ ЗАВЕРШЕНА 🔴\n"
                         f"Пользователь: {user_info_db[user_chat_id]['name']}\n"
                         f"ID: {user_chat_id}"
                )
                # Удаляем из активных консультаций
                del active_consultations[user_chat_id]
                if user_chat_id in user_info_db:
                    del user_info_db[user_chat_id]
            except Exception as e:
                print(f"Ошибка уведомления специалиста: {e}")
        
        await update.message.reply_text(
            "🗣️ Консультация завершена. Теперь вы общаетесь с ботом.\n"
            "Задайте ваш юридический вопрос!"
        )
    else:
        # Очищаем историю диалога с моделью
        if user_chat_id in user_dialog_history:
            del user_dialog_history[user_chat_id]
        await update.message.reply_text("🆕 Новый диалог начат. История очищена. Задайте Ваш вопрос.")

# Обработчик для сообщений от специалиста
async def handle_specialist_message(update: Update, context):
    specialist_chat_id = update.message.chat_id
    message_text = update.message.text
    
    # Проверяем команду ответа пользователю
    if message_text.startswith('/to_'):
        match = re.match(r'/to_(\d+)\s+(.*)', message_text, re.DOTALL)
        if match:
            try:
                user_chat_id = int(match.group(1))
                answer_text = match.group(2).strip()
                
                # Проверяем активность консультации
                if user_chat_id in active_consultations:
                    # Пересылаем сообщение пользователю
                    await context.bot.send_message(
                        chat_id=user_chat_id,
                        text=f"👨‍⚖️ Ответ от специалиста:\n\n{answer_text}"
                    )
                    # Подтверждаем специалисту
                    await update.message.reply_text(f"✅ Ответ отправлен пользователю {user_info_db[user_chat_id]['name']}")
                else:
                    await update.message.reply_text("❌ Консультация с этим пользователем не активна или завершена.")
            except (ValueError, IndexError):
                await update.message.reply_text("❌ Неверный формат команды. Используйте: /to_123456789 Ваш ответ")
        else:
            await update.message.reply_text("❌ Неверный формат команды. Используйте: /to_123456789 Ваш ответ")
    
    # Обработка других команд специалиста
    elif message_text.startswith('/list'):
        # Показать активные консультации
        if active_consultations:
            response = "📋 Активные консультации:\n\n"
            for user_id, spec_id in active_consultations.items():
                if spec_id == specialist_chat_id:
                    user_info = user_info_db.get(user_id, {"name": "Неизвестный", "username": "нет"})
                    response += f"👤 {user_info['name']} (@{user_info['username']})\n🆔 ID: {user_id}\n\n"
            await update.message.reply_text(response)
        else:
            await update.message.reply_text("ℹ️ Нет активных консультаций")
    
    elif message_text.startswith('/help'):
        # Справка по командам
        help_text = (
            "🛠️ Доступные команды:\n\n"
            "/to_123456789 [текст] - Ответить пользователю с указанным ID\n"
            "/list - Показать активные консультации\n"
            "/help - Эта справка"
        )
        await update.message.reply_text(help_text)
    
    else:
        # Обычное сообщение без команды
        if active_consultations:
            await update.message.reply_text(
                "ℹ️ Пожалуйста, используйте команды для работы:\n"
                "/help - показать справку\n"
                "/list - активные консультации"
            )
        else:
            await update.message.reply_text("ℹ️ Нет активных консультаций. Ожидайте новых запросов.")

# Обработчик сообщений от пользователей
async def handle_message(update: Update, context):
    user_chat_id = update.message.chat_id
    message_text = update.message.text
    
    # Игнорируем команды
    if message_text.startswith('/'):
        return
    
    if context.user_data.get('in_consultation', False):
        if user_chat_id in active_consultations:
            try:
                # Пересылаем сообщение специалисту
                await context.bot.send_message(
                    chat_id=active_consultations[user_chat_id],
                    text=f"✉️ Сообщение от {user_info_db[user_chat_id]['name']} (@{user_info_db[user_chat_id]['username']})\n"
                         f"🆔 ID: {user_chat_id}\n\n"
                         f"{message_text}"
                )
                await update.message.reply_text("✅ Ваше сообщение передано специалисту. Ожидайте ответа.")
            except Exception as e:
                print(f"Ошибка пересылки специалисту: {e}")
                await update.message.reply_text(
                    "❌ Не удалось отправить сообщение специалисту. "
                    "Попробуйте позже или завершите консультацию /newdialog"
                )
        else:
            await update.message.reply_text(
                "⚠️ Сессия консультации не активна. Используйте /consult для подключения."
            )
    else:
        # Создаем задачу для периодической отправки индикатора "печатает"
        typing_task = asyncio.create_task(
            send_continuous_typing(context.bot, user_chat_id)
        )
        
        # Создаем задачу для генерации ответа
        generate_task = asyncio.create_task(
            generate_response_async(message_text, user_chat_id)
        )
        
        
        try: #прямое ожидание ответа от модели
            answer = await generate_task
            await update.message.reply_text(answer)
        except Exception as e:
            print(f"Ошибка при генерации ответа: {e}")
            await update.message.reply_text("⚠️ Произошла ошибка при обработке вашего запроса. Пожалуйста, попробуйте еще раз.")
        finally:
            # Останавливаем все задачи
            typing_task.cancel()
            try:
                await typing_task
            except asyncio.CancelledError:
                pass

async def generate_response_async(message_text, user_chat_id):
    """Асинхронная обертка для generate_response"""
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(
        None, 
        generate_response, 
        message_text, 
        user_chat_id
    )

async def send_continuous_typing(bot, chat_id):
    """Периодически отправляет индикатор 'печатает' до отмены задачи"""
    try:
        while True:
            await bot.send_chat_action(chat_id=chat_id, action="typing")
            await asyncio.sleep(3)  # Отправляем каждые 3 секунды
    except asyncio.CancelledError:
        pass
    except Exception as e:
        print(f"Ошибка при отправке индикатора печати: {e}")




# MAIN
if __name__ == '__main__':
    TELEGRAM_TOKEN = "token" # ЗАМЕНИТЕ НА токен бота
    app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()
    
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("consult", start_consultation))
    app.add_handler(CommandHandler("newdialog", new_dialog))
    
    # Обработчики сообщений от пользователей
    app.add_handler(MessageHandler(
        filters.TEXT & ~filters.COMMAND,
        handle_message
    ))
    
    # Обработчик для специалиста
    app.add_handler(MessageHandler(
        filters.TEXT & filters.Chat(chat_id=int(SPECIALIST_CHAT_ID)),
        handle_specialist_message
    ))
    
    print("Бот запущен...")
    app.run_polling()